In [2]:
import polars as pl

INPUT_FILE = "../samples/gpu/9977228855307-r8937440-n976057.csv"

In [3]:
df = pl.read_csv(INPUT_FILE)
df


timestamp,gpu_index,utilization_gpu_pct,utilization_memory_pct,memory_free_MiB,memory_used_MiB,temperature_gpu,temperature_memory,power_draw_W,pcie_link_width_current
f64,i64,i64,i64,i64,i64,i64,i64,f64,i64
1.6214e9,0,0,0,32510,0,56,57,52.61,16
1.6214e9,0,0,0,32510,0,56,57,52.6,16
1.6214e9,0,0,0,32510,0,56,57,52.53,16
1.6214e9,0,0,0,32510,0,56,57,52.62,16
1.6214e9,0,0,0,32510,0,56,57,52.62,16
…,…,…,…,…,…,…,…,…,…
1.6214e9,0,100,6,32510,0,66,67,57.85,16
1.6214e9,0,14,1,32510,0,66,67,57.68,16
1.6214e9,0,14,1,32510,0,65,67,57.47,16


In [4]:
out_cols = [
    "utilization_gpu_pct",
    "utilization_memory_pct",
    "memory_free_MiB",
    "memory_used_MiB",
    "temperature_gpu",
    "temperature_memory",
    "power_draw_W",
    "pcie_link_width_current",
]
out_cols_means = list(map(lambda col_name: pl.col(col_name).mean(), out_cols))

df = df.with_columns(
pl.from_epoch(pl.col("timestamp"), time_unit="s").alias("datetime")
)
df = df.sort(by="datetime")
out_df = df.group_by_dynamic(index_column="datetime", every="1m", group_by="gpu_index").agg(out_cols_means)
out_df

gpu_index,datetime,utilization_gpu_pct,utilization_memory_pct,memory_free_MiB,memory_used_MiB,temperature_gpu,temperature_memory,power_draw_W,pcie_link_width_current
i64,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64
0,2021-05-19 18:02:00,0.0,0.0,32510.0,0.0,55.291339,55.708661,51.993228,16.0
0,2021-05-19 18:03:00,0.0,0.0,32510.0,0.0,50.903448,50.818966,49.703224,16.0
0,2021-05-19 18:04:00,0.0,0.0,32508.393103,1.606897,46.863793,46.232759,47.915121,16.0
0,2021-05-19 18:05:00,2.075862,0.686207,32009.618966,500.381034,45.125862,44.168966,51.251362,16.0
0,2021-05-19 18:06:00,84.565517,51.906897,4980.496552,27529.503448,59.822414,62.418966,183.49031,16.0
0,2021-05-19 18:07:00,83.237931,52.47931,4705.0,27805.0,70.105172,74.498276,187.974362,16.0
0,2021-05-19 18:08:00,83.181818,49.107438,7692.31405,24817.68595,70.578512,75.404959,187.281653,16.0


In [7]:
MODIFIED_FILE = "../samples/gpu/modified/9977228855307-r8937440-n976057-index-added.csv"

df_modified = pl.read_csv(MODIFIED_FILE)
df_modified = df_modified.with_columns(
pl.from_epoch(pl.col("timestamp"), time_unit="s").alias("datetime")
)
df_modified = df_modified.sort(by=["gpu_index","datetime"])
df_modified

timestamp,gpu_index,utilization_gpu_pct,utilization_memory_pct,memory_free_MiB,memory_used_MiB,temperature_gpu,temperature_memory,power_draw_W,pcie_link_width_current,datetime
f64,i64,i64,i64,i64,i64,i64,i64,f64,i64,datetime[μs]
1.6214e9,0,0,0,32510,0,56,57,52.61,16,2021-05-19 18:02:46
1.6214e9,0,0,0,32510,0,56,57,52.37,16,2021-05-19 18:02:48
1.6214e9,0,0,0,32510,0,56,57,52.58,16,2021-05-19 18:02:48
1.6214e9,0,0,0,32510,0,56,56,52.34,16,2021-05-19 18:02:48
1.6214e9,0,0,0,32510,0,56,56,52.46,16,2021-05-19 18:02:48
…,…,…,…,…,…,…,…,…,…,…
1.6214e9,1,0,0,32510,0,56,57,52.5,16,2021-05-19 18:02:47
1.6214e9,1,0,0,32510,0,56,57,52.39,16,2021-05-19 18:02:47
1.6214e9,1,0,0,32510,0,56,57,52.49,16,2021-05-19 18:02:47


In [10]:
out_df_modified = df_modified.group_by_dynamic(index_column="datetime", every="1m", group_by="gpu_index").agg(out_cols_means)
out_df_modified.write_csv("../sample.csv")